In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import db_helper
import datetime
import networkx as nx
from datetime import datetime
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import importlib
import graph_constructor as GC
import n2v as N2V
import utils
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [145]:
import gc
gc.collect()

33958313

In [4]:
PROJECT_PATH='/home/hmenn/Workspace/CSE496_Node2Vec_2018'
MONTH_TAG_EN="month"
YEAR_TAG_EN="year"
DAY_TAG_EN="day"

MONTH_NAMES_EN = {1:"JAN",2:"FEB",3:"MAR",4:"APR",
          5:"MAY",6:"JUN",7:"JUL",8:"AUG",
         9:"SEP",10:"OCT",11:"NOV",12:"DEC"}

MONTH_TAG_TR="AY"
YEAR_TAG_TR="YIL"
DAY_TAG_TR="GUN"

MONTH_NAMES_TR = {1:"OCA",2:"SUB",3:"MAR",4:"NIS",
          5:"MAY",6:"HAZ",7:"TEM",8:"AGU",
         9:"EYL",10:"EKI",11:"KAS",12:"ARA"}

In [28]:
db = db_helper.DBHelper()
db.connectMysql(user="root",passwd="Hasan5695*",db="SocialMediaDB")
tweets = db.getTweetsFromMysql("TweetsShort")
db_helper.saveTweetsAsJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json', tweets)

In [5]:
tweets = db_helper.getTweetsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json')

In [21]:
allFriendships = db_helper.getAllFriendsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/friends.json')

In [136]:
print('TotalTweet#:',len(tweets))
print(tweets[:2])

TotalTweet#: 93753
[############ Tweet: ############ 
SendByScreenName:cakrr_ 
UserID:u1623755515 
TweetID:t763141405766250496 
InReplyToStatusID:t763141168771244032 
InReplyToUserID:u757176735158923264 
Date:2016-08-10 01:34:24 
Text:@Wesleyoloji10 OGLAK HAZİNE ARIYOR AQ:das:asd 
, ############ Tweet: ############ 
SendByScreenName:selcukarici 
UserID:u154236788 
TweetID:t763577912696135680 
InReplyToStatusID:t-1 
InReplyToUserID:u-1 
Date:2016-08-11 06:28:55 
Text:Sn.CB'nımızın çağrısıyla 3 banka konut kredisi faiz oranlarını ℅1'in altına çekti..Bu olumlu bir adım.

Ancak şişen piyasa nasıl düşecek? 
]


In [131]:
print(allFriendships[:2])
userFriends = [('u100341775',x['FriendID'], {'weight':1}) for x in allFriendships[:] if x['UserID']=='u100341775']
print(userFriends)

[{'UserID': 'u100341775', 'FriendID': 'u1014141614'}, {'UserID': 'u100341775', 'FriendID': 'u120840897'}]
[('u100341775', 'u1014141614', {'weight': 1}), ('u100341775', 'u120840897', {'weight': 1}), ('u100341775', 'u1494034783', {'weight': 1}), ('u100341775', 'u154236788', {'weight': 1}), ('u100341775', 'u155534196', {'weight': 1}), ('u100341775', 'u190171522', {'weight': 1}), ('u100341775', 'u219248343', {'weight': 1}), ('u100341775', 'u2547943374', {'weight': 1}), ('u100341775', 'u25506738', {'weight': 1}), ('u100341775', 'u272613386', {'weight': 1}), ('u100341775', 'u285685164', {'weight': 1}), ('u100341775', 'u3228821009', {'weight': 1}), ('u100341775', 'u41931916', {'weight': 1}), ('u100341775', 'u425819395', {'weight': 1}), ('u100341775', 'u461858710', {'weight': 1}), ('u100341775', 'u497735542', {'weight': 1}), ('u100341775', 'u61189897', {'weight': 1}), ('u100341775', 'u63110096', {'weight': 1}), ('u100341775', 'u737302248338952193', {'weight': 1})]


In [142]:
def create_all_graph(tweets):
    G1 = nx.DiGraph()
    G1.add_node(YEAR_TAG_TR)
    G1.add_node(MONTH_TAG_TR)
    G1.add_node(DAY_TAG_TR)

    prevYear = None
    prevMonth = None
    prevDay = None

    prevYear = tweets[0].date.year
    prevMonth = tweets[0].date.month
    prevDay = tweets[0].date.day

    for tweet in tweets:
        G1.add_edge(YEAR_TAG_TR,str(tweet.date.year),weight=1.0)
        
        if(prevYear < tweet.date.year):
            G1.add_edge(str(prevYear),str(tweet.date.year),weight=1.0)
            prevYear = tweet.date.year
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
          
        monthFormat = "{month}.{year}".format(month=MONTH_NAMES_TR[tweet.date.month],year=tweet.date.year)
        G1.add_edge(str(tweet.date.year),monthFormat,weight=1.0)
        
        if(prevMonth < tweet.date.month):
            prevMonthFormat = "{month}.{year}".format(month=MONTH_NAMES_TR[prevMonth],year=tweet.date.year)
            G1.add_edge(prevMonthFormat,monthFormat,weight=1.0)
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
        
        #G1.add_edge(MONTH_TAG,monthFormat,weight=1.0)
        fullDateFormat = "{day}.{month}.{year}".format(day=tweet.date.day,month=tweet.date.month,year=tweet.date.year)
        if prevDay < tweet.date.day:
            prevFullDateFormat = "{day}.{month}.{year}".format(day=prevDay,month=tweet.date.month,year=tweet.date.year)
            G1.add_edge(prevFullDateFormat,fullDateFormat,weight=1.0)
            prevDay = tweet.date.day
        
        # tweet -> date
        G1.add_edge(tweet.tweetID,fullDateFormat,weight=1.0)
        G1.add_edge(monthFormat,fullDateFormat,weight=1.0)
                
        # if user will be added first time, add friend connections
        if not G1.has_node(tweet.userID):
            userFriends = [(x['UserID'],x['FriendID'], {'weight':1.0}) for x in allFriendships if x['UserID']==tweet.userID]
            #print('User:{userID} was added. User has {friendCount} friend.'.format(userID=tweet.userID,friendCount=len(userFriends)))

            # add friendship connections
            G1.add_edges_from(userFriends)
            
        # user -> tweet connections were established
        G1.add_edge(tweet.userID, tweet.tweetID,weight=1.0)
                 
        # if tweet replied and db has actuaal(replied tweet, add to graph)
        # cevap verilen tweet db de ise, o tweete baglanti ekle
        for i in tweets:
            if i.tweetID == tweet.inReplyToStatusID:
                G1.add_edge(tweet.tweetID, tweet.inReplyToStatusID,weight=1.0)

    return G1

In [143]:
startTime = datetime.now()
graph = create_all_graph(tweets[:3000])
endTime = datetime.now()
print("Graph construct time: ",endTime-startTime)
print("Number of edges: ",graph.number_of_edges())
print("Number of nodes: ",graph.number_of_nodes())

Graph construct time:  0:00:00.884409
Number of edges:  6582
Number of nodes:  3259


In [138]:
# save and read edge list
#nx.write_edgelist(graph, path=PROJECT_PATH+"/outputs/grid.edgelist", delimiter=":")
#graph = nx.read_edgelist(path="grid.edgelist", delimiter=":")

In [140]:
#graph.edges()
#graph['u100341775']

In [146]:
startTime = datetime.now()
graphN2V = N2V.Graph(graph, True, 1.0, 1.0)
graphN2V.preprocess_transition_probs()
walks = graphN2V.simulate_walks(3000, 80)
endTime = datetime.now()
print("Node2Vec simulation walk time: ",endTime-startTime)

Walk iteration:
500 / 3000
1000 / 3000
1500 / 3000
2000 / 3000
2500 / 3000
3000 / 3000
Node2Vec simulation walk time:  0:08:02.394668


In [147]:
startTime = datetime.now()
model = N2V.learn_embeddings(walks,"graphAll")
endTime = datetime.now()
print("Learning embeddings time: ",endTime-startTime)

Learning embeddings time:  0:03:49.195420


In [148]:
model.wv.most_similar('u1623755515')

[('u41931916', 0.6401066780090332),
 ('u757176735158923264', 0.6349278092384338),
 ('u819201027857641472', 0.6320477724075317),
 ('u760134507072786432', 0.6316286325454712),
 ('u808341696715096064', 0.6294573545455933),
 ('u82946124', 0.627635657787323),
 ('u838307023527768064', 0.6272684931755066),
 ('u875668493768810496', 0.6268239617347717),
 ('u1014141614', 0.625278115272522),
 ('u292822200', 0.6238027215003967)]

In [149]:
column_names = ['2016', 'AGU.2016', '24.8.2016', "20.8.2016","t766739153195266049","t766738735786491905", "t766744289342808069",'u1623755515','u4242878596','u2335496184']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,2016,AGU.2016,24.8.2016,20.8.2016,t766739153195266049,t766738735786491905,t766744289342808069,u1623755515,u4242878596,u2335496184
2016,1.000000,0.897912,0.089368,0.147665,0.248144,0.218022,0.111865,0.181527,0.298479,0.123207
AGU.2016,0.897912,1.000000,0.180982,0.230807,0.318939,0.285151,0.140732,0.130108,0.260527,0.223532
24.8.2016,0.089368,0.180982,1.000000,0.329451,0.237361,0.213374,0.197683,0.058424,0.190358,0.233612
20.8.2016,0.147665,0.230807,0.329451,1.000000,0.256564,0.240994,0.131919,0.056933,0.196134,0.193339
t766739153195266049,0.248144,0.318939,0.237361,0.256564,1.000000,0.948233,0.431832,0.100725,0.236257,0.511241
t766738735786491905,0.218022,0.285151,0.213374,0.240994,0.948233,1.000000,0.423794,0.075859,0.217098,0.495944
t766744289342808069,0.111865,0.140732,0.197683,0.131919,0.431832,0.423794,1.000000,-0.076573,0.201869,0.460607
u1623755515,0.181527,0.130108,0.058424,0.056933,0.100725,0.075859,-0.076573,1.000000,0.617314,0.073153
u4242878596,0.298479,0.260527,0.190358,0.196134,0.236257,0.217098,0.201869,0.617314,1.000000,0.306715
u2335496184,0.123207,0.223532,0.233612,0.193339,0.511241,0.495944,0.460607,0.073153,0.306715,1.000000
